In [17]:
import os
Root = "Audio_Speech_Actors_01-24"
os.chdir(Root)

In [18]:
ls

Actor_01/ Actor_04/ Actor_07/ Actor_10/ Actor_13/ Actor_16/ Actor_19/ Actor_22/
Actor_02/ Actor_05/ Actor_08/ Actor_11/ Actor_14/ Actor_17/ Actor_20/ Actor_23/
Actor_03/ Actor_06/ Actor_09/ Actor_12/ Actor_15/ Actor_18/ Actor_21/ Actor_24/


In [32]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import audioread



In [34]:
# Define function to extract features from audio file
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    X, sr = librosa.load(file_name, sr=None)  # Load audio file with its sampling rate
    result = []
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T, axis=0)
        result.append(mfccs)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=X, sr=sr).T, axis=0)
        result.append(chroma)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sr).T, axis=0)
        result.append(mel)
    return np.hstack(result) if len(result) > 0 else None

In [21]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [22]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [35]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [36]:
x_train

array([[-6.6763867e+02,  6.8609329e+01,  5.3894248e+00, ...,
         1.2315154e-07,  1.4720987e-08,  7.4151187e-09],
       [-4.6218417e+02,  5.8300858e+01, -1.7537727e+01, ...,
         3.7088807e-06,  2.9893465e-07,  1.4202649e-08],
       [-6.8122150e+02,  7.8883621e+01,  1.2867522e-01, ...,
         4.2481201e-08,  1.1203549e-08,  7.8892199e-09],
       ...,
       [-7.0643005e+02,  9.5198959e+01,  4.8495064e+00, ...,
         1.3814628e-08,  6.9739534e-09,  6.5839383e-09],
       [-6.9807587e+02,  7.0690025e+01, -2.2900000e+00, ...,
         1.9329592e-08,  7.7685023e-09,  6.1271024e-09],
       [-7.4504138e+02,  7.4680641e+01,  2.9166033e+00, ...,
         5.2143715e-08,  1.3141310e-08,  7.8580920e-09]], dtype=float32)

In [37]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [38]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [39]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [40]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [41]:
#Predict for the test set
y_pred=model.predict(x_test)

In [42]:
y_pred

array(['fearful', 'fearful', 'disgust', 'disgust', 'fearful', 'fearful',
       'fearful', 'fearful', 'disgust', 'calm', 'calm', 'fearful',
       'disgust', 'calm', 'happy', 'disgust', 'fearful', 'happy',
       'disgust', 'fearful', 'happy', 'fearful', 'fearful', 'calm',
       'fearful', 'happy', 'fearful', 'calm', 'fearful', 'happy', 'happy',
       'fearful', 'disgust', 'fearful', 'calm', 'calm', 'calm', 'happy',
       'fearful', 'happy', 'happy', 'calm', 'calm', 'fearful', 'fearful',
       'fearful', 'calm', 'happy', 'calm', 'happy', 'fearful', 'happy',
       'disgust', 'happy', 'fearful', 'disgust', 'calm', 'disgust',
       'calm', 'calm', 'happy', 'fearful', 'calm', 'fearful', 'fearful',
       'fearful', 'disgust', 'calm', 'happy', 'fearful', 'fearful',
       'calm', 'happy', 'fearful', 'calm', 'disgust', 'calm', 'calm',
       'disgust', 'fearful', 'disgust', 'fearful', 'fearful', 'disgust',
       'calm', 'happy', 'calm', 'calm', 'fearful', 'fearful', 'calm',
       'ha

In [43]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 61.98%


In [44]:
from sklearn.metrics import accuracy_score, f1_score

In [45]:
f1_score(y_test, y_pred,average=None)

array([0.68041237, 0.61538462, 0.57894737, 0.6097561 ])

In [46]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,fearful
1,fearful,fearful
2,disgust,disgust
3,disgust,disgust
4,happy,fearful
5,fearful,fearful
6,calm,fearful
7,happy,fearful
8,disgust,disgust
9,calm,calm


In [47]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [51]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/Users/manjushreemuralidhara/Masters/HCI/Speech-Emotion-Analyzer/Audio_Speech_Actors_01-24/Actor_01/03-01-02-02-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [52]:
import librosa
import numpy as np
import pickle

class SpeechEmotionAnalyzer:
    def __init__(self, model_filename):
        self.model_filename = model_filename
        self.loaded_model = self.load_model()

    def load_model(self):
        return pickle.load(open(self.model_filename, 'rb'))

    def extract_feature(self, audio_file, mfcc=True, chroma=True, mel=True):
        X, sr = librosa.load(audio_file, sr=None)  # Load audio file with its sampling rate
        result = []
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T, axis=0)
            result.append(mfccs)
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(y=X, sr=sr).T, axis=0)
            result.append(chroma)
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sr).T, axis=0)
            result.append(mel)
        return np.hstack(result) if len(result) > 0 else None

    def predict_emotion(self, audio_file):
        feature = self.extract_feature(audio_file, mfcc=True, chroma=True, mel=True)
        if feature is not None:
            feature = feature.reshape(1, -1)
            prediction = self.loaded_model.predict(feature)
            return prediction[0]  # Return the predicted emotion label
        else:
            return "Error: Unable to extract features from audio file."


In [53]:
filename = 'modelForPrediction1.sav'
analyzer = SpeechEmotionAnalyzer(filename)
prediction = analyzer.predict_emotion("/Users/manjushreemuralidhara/Masters/HCI/Speech-Emotion-Analyzer/Audio_Speech_Actors_01-24/Actor_01/03-01-02-02-01-01-01.wav")
print(prediction)

calm
